In [ ]:
from google.colab import drive
import os

drive.mount('/content/drive')

input_folder = '/content/drive/MyDrive/ai_techwiz_2/data'
cropped_folder = '/content/drive/MyDrive/ai_techwiz_2/face_dataset_cropped'
embeddings_folder = '/content/drive/MyDrive/ai_techwiz_2/face_embeddings'

os.makedirs(input_folder, exist_ok=True)
os.makedirs(cropped_folder, exist_ok=True)
os.makedirs(embeddings_folder, exist_ok=True)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install facenet-pytorch


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 71.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 65.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 77.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 819.4 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 51.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 30.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.

In [ ]:
!pip install hnswlib


In [ ]:
!pip install faiss-cpu


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 31.1 MB/s eta 0:00:00


In [ ]:

import os
import torch
import numpy as np
from PIL import Image, ImageFilter, ImageOps
from facenet_pytorch import MTCNN, InceptionResnetV1, extract_face, fixed_image_standardization
from tqdm import tqdm
import joblib
import cv2


# **Data preprocessing**

In [ ]:

class FaceRecognitionConfig:
    DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    IMAGE_SIZE = 160
    MARGIN = 20
    MIN_FACE_SIZE = 20
    THRESHOLDS = [0.6, 0.7, 0.7]
    BATCH_SIZE = 32

def ensure_dir(path: str):
    os.makedirs(path, exist_ok=True)



In [ ]:

def get_mtcnn(device=FaceRecognitionConfig.DEVICE, image_size=160, margin=20) -> MTCNN:
    return MTCNN(
        image_size=image_size,
        margin=margin,
        min_face_size=FaceRecognitionConfig.MIN_FACE_SIZE,
        thresholds=FaceRecognitionConfig.THRESHOLDS,
        factor=0.709,
        post_process=False,
        device=device,
        keep_all=False,
        select_largest=True
    )

def get_resnet(device=FaceRecognitionConfig.DEVICE, pretrained='vggface2') -> InceptionResnetV1:
    return InceptionResnetV1(pretrained=pretrained).eval().to(device)

mtcnn = get_mtcnn()
resnet = get_resnet()
print("Models ready. Device:", FaceRecognitionConfig.DEVICE)


  0%|          | 0.00/107M [00:00<?, ?B/s]

Models ready. Device: cpu


## **Face Detection**

In [ ]:
import torch

def process_dataset(input_root: str, output_root: str, mtcnn: MTCNN = None):

    if mtcnn is None:
        mtcnn = get_mtcnn()

    ensure_dir(output_root)
    persons = [d for d in os.listdir(input_root)
               if os.path.isdir(os.path.join(input_root, d)) and not d.startswith('.')]

    for person in persons:
        person_input_dir = os.path.join(input_root, person)
        person_output_dir = os.path.join(output_root, person)
        ensure_dir(person_output_dir)

        img_list = [f for f in os.listdir(person_input_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

        for img_name in tqdm(img_list, desc=f"Processing {person}", ncols=100):
            img_path = os.path.join(person_input_dir, img_name)
            pil_img = Image.open(img_path).convert('RGB')

            face, prob = mtcnn(pil_img, return_prob=True)
            if face is None:
                print(f"Không tìm thấy mặt trong ảnh {img_path}")
                continue
            ensure_dir(person_output_dir)

            tensor_path = os.path.join(person_output_dir, img_name.replace(".png", ".pt"))
            torch.save(face, tensor_path)

    print("Xử lý dataset xong (lưu tensor)!")


process_dataset(input_folder, cropped_folder, mtcnn)

Processing Nguyen_Thi_Hong_Mai:  69%|████████████████████▋         | 99/144 [00:24<00:08,  5.38it/s]

Không tìm thấy mặt trong ảnh /content/drive/MyDrive/ai_techwiz_2/data/Nguyen_Thi_Hong_Mai/SV_0011_106.png


Processing Nguyen_Thi_Cam_Ly:  89%|█████████████████████████████▍   | 65/73 [00:15<00:01,  5.42it/s]

Không tìm thấy mặt trong ảnh /content/drive/MyDrive/ai_techwiz_2/data/Nguyen_Thi_Cam_Ly/SV_0010_069.png
Không tìm thấy mặt trong ảnh /content/drive/MyDrive/ai_techwiz_2/data/Nguyen_Thi_Cam_Ly/SV_0010_070.png


Processing Nguyen_Thi_Cam_Ly:  95%|███████████████████████████████▏ | 69/73 [00:16<00:00,  5.78it/s]

Không tìm thấy mặt trong ảnh /content/drive/MyDrive/ai_techwiz_2/data/Nguyen_Thi_Cam_Ly/SV_0010_068.png


Processing Nguyen_Thi_Cam_Ly: 100%|█████████████████████████████████| 73/73 [00:17<00:00,  4.29it/s]


Không tìm thấy mặt trong ảnh /content/drive/MyDrive/ai_techwiz_2/data/Nguyen_Thi_Cam_Ly/SV_0010_067.png
Không tìm thấy mặt trong ảnh /content/drive/MyDrive/ai_techwiz_2/data/Nguyen_Thi_Cam_Ly/SV_0010_071.png


Processing Mai_Thanh_Thu: 100%|█████████████████████████████████████| 91/91 [00:36<00:00,  2.51it/s]

Xử lý dataset xong (lưu tensor)!


## **Face Embedding**

In [ ]:
NAME_TO_MSV = {
    "Nguyen_Van_Tuan": "SV_0007",
    "Nguyen_Viet_Quoc_An": "SV_0008",
    "Nguyen_Van_Minh": "SV_0006",
    "Nguyen_Thi_Phuong_Thao": "SV_0013",
    "Nguyen_Thi_Hong_Mai": "SV_0011",
    "Nguyen_Thi_Cam_Ly": "SV_0010",
    "Nguyen_The_Truong": "SV_0009",
    "Nguyen_Phu_Nguyen": "SV_0005",
    "Nguyen_Phong_Hai": "SV_0004",
    "Nguyen_Ha_Phuong_Uyen": "SV_0012",
    "Nguyen_Duy_Hoang": "SV_0001",
    "Nguyen_Duc_Phong": "SV_0002",
    "Mai_Thanh_Thu": "SV_0014",
    "Le_Duc_Nguyen": "SV_0003"

}


In [ ]:
import os
import numpy as np
import torch
from tqdm import tqdm
from PIL import Image

def pil_to_tensor_standard(pil_img: Image.Image) -> torch.Tensor:

    arr = np.asarray(pil_img).astype(np.float32) / 255.0
    t = torch.from_numpy(arr.transpose(2, 0, 1)).float()
    return fixed_image_standardization(t)


def compute_embeddings_all(
    cropped_root: str,
    embeddings_root: str,
    resnet: InceptionResnetV1 = None,
    batch_size: int = 32,
    name_to_msv: dict = None
):
    if resnet is None:
        resnet = get_resnet()

    registry = {}

    persons = [d for d in os.listdir(cropped_root) if os.path.isdir(os.path.join(cropped_root, d))]

    for person in tqdm(persons, desc="Compute embeddings (all)"):
        person_dir = os.path.join(cropped_root, person)
        tensor_files = [f for f in os.listdir(person_dir) if f.lower().endswith('.pt')]

        if not tensor_files:
            continue

        face_tensors = []
        for tf in tensor_files:
            tensor_path = os.path.join(person_dir, tf)
            face = torch.load(tensor_path)              # [3,H,W], float [0,1]
            face = fixed_image_standardization(face)    # [-1,1]
            face_tensors.append(face)

        face_stack = torch.stack(face_tensors)  # [N,3,H,W]


        all_embeddings = []
        with torch.no_grad():
            for i in range(0, len(face_stack), batch_size):
                batch = face_stack[i:i+batch_size]
                emb = resnet(batch)  # [B,512]
                all_embeddings.append(emb)
        all_embeddings = torch.cat(all_embeddings, dim=0)  # [N,512]

        student_id = name_to_msv.get(person, person) if name_to_msv else person

        registry[student_id] = all_embeddings.cpu().numpy()

    os.makedirs(embeddings_root, exist_ok=True)
    np.savez(os.path.join(embeddings_root, "embeddings_all.npz"), **registry)


compute_embeddings_all(cropped_folder, embeddings_folder, resnet, name_to_msv=NAME_TO_MSV)

# **Train/test split**

In [ ]:
embeddings_data = '/content/drive/MyDrive/ai_techwiz_2/face_embeddings'
data = np.load(os.path.join(embeddings_data, "embeddings_all.npz"))
dict1 = {k: data[k] for k in data.files}

In [ ]:
dict1.keys()

dict_keys(['SV_0007', 'SV_0008', 'SV_0003', 'SV_0004', 'SV_0001', 'SV_0002', 'SV_0005', 'SV_0006', 'SV_0012', 'SV_0009', 'SV_0011', 'SV_0010', 'SV_0013', 'SV_0014'])

In [ ]:
sorted_dict = dict(sorted(dict1.items()))
sorted_dict.keys()

dict_keys(['SV_0001', 'SV_0002', 'SV_0003', 'SV_0004', 'SV_0005', 'SV_0006', 'SV_0007', 'SV_0008', 'SV_0009', 'SV_0010', 'SV_0011', 'SV_0012', 'SV_0013', 'SV_0014'])

In [ ]:
from sklearn.model_selection import train_test_split

data = sorted_dict
X, y = [], []

label_map, reverse_map = {}, {}

label_counter = 1

for student_id, vectors in data.items():
    if student_id not in label_map:
        label_map[student_id] = label_counter
        reverse_map[label_counter] = student_id
        label_counter += 1
    for v in vectors:
        X.append(v)
        y.append(label_map[student_id])

X = np.array(X).astype('float32')
y = np.array(y).astype('int')

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# **Cosine**

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import accuracy_score

y_pred = []
for x in X_test:
    sims = cosine_similarity([x], X_train)[0]
    idx_max = np.argmax(sims)
    y_pred.append(y_train[idx_max])

acc = accuracy_score(y_test, y_pred)
print("Accuracy:", acc)

Accuracy: 1.0


# **Model: SVM**

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score

svm = SVC(kernel="linear", probability=True, random_state=42)
svm.fit(X_train, y_train)

y_pred = svm.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print("Accuracy:", acc)
print(classification_report(y_test, y_pred))


Accuracy: 1.0
              precision    recall  f1-score   support

           1       1.00      1.00      1.00        20
           2       1.00      1.00      1.00        19
           3       1.00      1.00      1.00        20
           4       1.00      1.00      1.00        20
           5       1.00      1.00      1.00        21
           6       1.00      1.00      1.00        20
           7       1.00      1.00      1.00        21
           8       1.00      1.00      1.00        20
           9       1.00      1.00      1.00        18
          10       1.00      1.00      1.00        14
          11       1.00      1.00      1.00        29
          12       1.00      1.00      1.00        22
          13       1.00      1.00      1.00        19
          14       1.00      1.00      1.00        18

    accuracy                           1.00       281
   macro avg       1.00      1.00      1.00       281
weighted avg       1.00      1.00      1.00       281



#**Model: HNSW**

In [ ]:
import hnswlib

dim = X.shape[1]
num_elements = X_train.shape[0]

index = hnswlib.Index(space='cosine', dim=dim)
index.init_index(max_elements=200000, ef_construction=200, M=32)
index.add_items(X_train, y_train)
index.set_ef(200)

labels, distances = index.knn_query(X_test, k=1)

y_pred = []
for label, dist in zip(labels[:,0], distances[:,0]):
    if dist > 0.3:
        y_pred.append(0)
    else:
        y_pred.append(label)

acc = accuracy_score(y_test, y_pred)
print("Accuracy:", acc)

Accuracy: 0.9679715302491103


# **Model: FAISS**

In [ ]:
import faiss

dim = X_train.shape[1]

index = faiss.IndexFlatL2(dim)
index.add(X_train)

def predict_one_faiss_k1(index, x):
    x = x.reshape(1, -1).astype('float32')
    D, I = index.search(x, 1)
    idx = I[0][0]
    dist = D[0][0]
    label = y_train[idx]

    similarity = 1 / (1 + dist)
    return label, similarity

y_pred = []
for x in X_test:
    lbl, _ = predict_one_faiss_k1(index, x)
    y_pred.append(lbl)

acc = accuracy_score(y_test, y_pred)
print("Accuracy FAISS Flat:", acc)


Accuracy FAISS Flat: 1.0


## Download model

In [ ]:
import pickle

faiss_path = "/content/drive/MyDrive/ai_techwiz_2/faiss_index.index"
labels_path = "/content/drive/MyDrive/ai_techwiz_2/faiss_labels.pkl"

faiss.write_index(index, faiss_path)

with open(labels_path, "wb") as f:
    pickle.dump(y_train, f)